In [1]:
import sqlite3 as sqlite
import pandas as pd

In [9]:
chembl_query_results = pd.read_csv('/scratch/ias41/FAERS_y3/ae_target_links/data/faers_sider_chembl_processed.txt', sep = '\t')

/usr/local/shared/anaconda/python3/5.3.1/lib/python3.7/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [10]:
chembl_query_results_human = chembl_query_results.loc[chembl_query_results['target_organism']=='Homo sapiens',]

In [11]:
result_molregnos = list(chembl_query_results_human['parent_molregno'].drop_duplicates())

In [12]:
# Open sqlite database with 
conn = sqlite.connect('/scratch/ias41/FAERS_y3/compound_mapping/output/201903_mapped_compounds_calculon.db')
cur = conn.cursor()

In [13]:
parent_query = """
select distinct aeolus_concept
from compound_structures
where mapped_parent_molregno in ({})""".format(', '.join([str(i) for i in result_molregnos]))

In [14]:
aeolus_concepts_for_ae_detection = [i[0] for i in cur.execute(parent_query).fetchall()]

In [15]:
print(aeolus_concepts_for_ae_detection[:10])
len(aeolus_concepts_for_ae_detection)

[700253, 700299, 700465, 701322, 702685, 702774, 702865, 703083, 703244, 703547]


1348

In [17]:
with open('/scratch/ias41/ae_code/psm_aeolus/data/aeolus_ids_with_bioact_for_ae_detection.txt', 'w') as f:
    f.write('\n'.join(str(i) for i in aeolus_concepts_for_ae_detection))

In [18]:
conn.close()

In [25]:
# Make file with rest to run
with open('/scratch/ias41/ae_code/psm_aeolus/data/aeolus_ids_with_bioact_for_ae_detection.txt', 'r') as f:
    drugs_w_bioact = [int(i) for i in f.read().split('\n')]

In [29]:
# Make file with rest to run
with open('/scratch/ias41/ae_code/psm_aeolus/data/aeolus_ids_for_ae_detection.txt', 'r') as f:
    all_drugs = [int(i) for i in f.read().split('\n')]

In [30]:
len(drugs_w_bioact), len(all_drugs)

(1348, 3526)

In [31]:
rest = list(set(drugs_wo_bioact) - set(drugs_w_bioact))

In [32]:
len(rest)

2178

In [34]:
with open('/scratch/ias41/ae_code/psm_aeolus/data/aeolus_ids_wo_bioact.txt', 'w') as f:
    f.write('\n'.join(str(i) for i in rest))